In [19]:
!pip install streamlit pyngrok --quiet

In [20]:
!ngrok config add-authtoken WE2SPUJNXCPUEAVTICCJHKXQONGVHJ3Q


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [28]:
!ngrok config add-authtoken 2vwIQrwjPio3L9KC2DeYe8vtlW4_5tzPNHZ6iG3BfrhojeWbK


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [31]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd

st.set_page_config(page_title="Live AQI Dashboard - WAQI", layout="centered")
st.title("🌍 Live AQI Dashboard using WAQI API")

city = st.text_input("Enter a City or Postcode", "EH49 6HR")

WAQI_TOKEN = "09591e540fc00070ffdf88e92ad50713a635a11a"

def fetch_waqi_data(city_name):
    url = f"https://api.waqi.info/feed/{city_name}/?token={WAQI_TOKEN}"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

def categorize_aqi(aqi):
    if aqi <= 50:
        return "🟢 Good"
    elif aqi <= 100:
        return "🟡 Moderate"
    elif aqi <= 150:
        return "🟠 Unhealthy for Sensitive Groups"
    elif aqi <= 200:
        return "🔴 Unhealthy"
    elif aqi <= 300:
        return "🟣 Very Unhealthy"
    else:
        return "⚫ Hazardous"

data = fetch_waqi_data(city)

if data and data["status"] == "ok":
    aqi = data["data"]["aqi"]
    dom_pollutant = data["data"].get("dominentpol", "N/A")
    iaqi = data["data"]["iaqi"]
    city_name = data["data"]["city"]["name"]
    time = data["data"]["time"]["s"]

    st.subheader(f"📍 Location: {city_name}")
    st.write(f"🕒 Time: {time}")
    st.metric(label="AQI", value=aqi, delta=categorize_aqi(aqi))
    st.caption(f"Dominant Pollutant: **{dom_pollutant.upper()}**")

    pollutants = {}
    for key in ['pm25', 'pm10', 'co', 'no2', 'so2', 'o3', 't', 'h']:
        if key in iaqi:
            pollutants[key.upper()] = iaqi[key]['v']

    df_pollutants = pd.DataFrame(pollutants.items(), columns=["Pollutant", "Value"])
    st.write("### 🌫️ Pollutant Levels")
    st.table(df_pollutants)
else:
    st.error("❌ Failed to fetch data from WAQI API. Try a valid city or postcode.")


Overwriting app.py


In [32]:
from pyngrok import ngrok
import threading
import time
import os

# Kill any existing tunnels
ngrok.kill()

# Start Streamlit in background
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)

# ✅ Fixed for ngrok v3 (use full URL)
public_url = ngrok.connect("http://localhost:8501")
print(f"🚀 Your Streamlit AQI dashboard is live at: {public_url}")


🚀 Your Streamlit AQI dashboard is live at: NgrokTunnel: "https://2346-34-118-240-209.ngrok-free.app" -> "http://localhost:8501"
